In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        return output

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

# 使用示例
node_features = torch.rand(10, 5)  # 10个节点，每个节点5个特征
adjacency_matrix = torch.eye(10)   # 邻接矩阵 (此处为单位矩阵，表示无边)
labels = torch.LongTensor([0, 1, 0, 1, 1, 0, 0, 1, 0, 1])  # 每个节点的标签

gcn = GCN(nfeat=node_features.shape[1], nhid=16, nclass=2, dropout=0.5)
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)

def train():
    gcn.train()
    optimizer.zero_grad()
    output = gcn(node_features, adjacency_matrix)
    loss = F.nll_loss(output, labels)
    loss.backward()
    optimizer.step()

# 训练10个epoch
for epoch in range(10):
    train()
